In [ ]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [ ]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + "="+ layer.__class__.__name__, layer.trainable)

In [ ]:
model.layers

In [ ]:
model.layers[0].input

In [ ]:
model.output

In [ ]:
# Here we freeze all the layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
def addTopModel(bottom_model, num_classes, neurons):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(neurons, activation = "relu")(top_model)
    top_model = Dropout(0.1)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2
neurons=512
FC_Head = addTopModel(model, num_classes, neurons)

newModel = Model(inputs=model.input, outputs=FC_Head)

print(newModel.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'vgg16/train/'
validation_data_dir = 'vgg16/test/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 5
 
train_set = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
test_set = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("vgg16.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 4,
                          verbose = 1,
                          restore_best_weights = True)


# Note we use a very small learning rate 
newModel.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

train_samples = 198
test_samples = 30
batch_size =8

history = newModel.fit_generator(
    train_set,
    steps_per_epoch = train_samples // batch_size,
    epochs = 5,
    callbacks = [earlystop, checkpoint],
    validation_data = test_set,
    validation_steps = test_samples // batch_size)

newModel.save("vgg16.h5")

In [2]:
from keras.models import load_model
model = load_model('vgg16.h5')

In [3]:
import cv2
import numpy as np
from PIL import Image

In [4]:
from  keras.preprocessing import image
face_cascade = cv2.CascadeClassifier('F://haarcascade_frontalface_default.xml')

In [5]:
def face_detect(img):
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        crop = img[y:y+h, x:x+w]
    return crop


In [23]:
cap = cv2.VideoCapture(0)
while True:
    _, frame = cap.read()
    face = face_detect(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        img_array = Image.fromarray(face, 'RGB')
        img_array = np.array(img_array)
        img_array = np.expand_dims(img_array, axis=0)
        result= model.predict(img_array)   
      
        
        if(result[0][1]>0.8):
            name="milind"
        elif(result[0][0]>0.8):
            name="jyoti"
        else:
            pass
        cv2.putText(frame,name,(50,50), cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            
    else:
        cv2.putText(frame,"Nothing found", (50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    cv2.imshow('video',frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
    
cap.release()
cv2.destroyAllWindows()  



In [24]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

face={"[0]": "jyoti", 
           "[1]": "milind",
            }
face_name={"jyoti":"jyoti",
           "milind":"milind"
          }
            
def draw_test(name, pred, im):
    Face = face[str(pred)]
    BLACK = [0,0,0]
    #expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(im, Face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, im)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + face_name[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(5):
    input_im = getRandomImage("vgg16/test/") 
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(model.predict(input_im, 1, verbose = 0), axis=1)
   
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)
print(res)
cv2.destroyAllWindows()

Class - milind
Class - milind
Class - jyoti
Class - jyoti
Class - jyoti
[0]


In [ ]:
cv2.destroy

In [ ]:
cv2.destroyAllWindows()